In [ ]:
%matplotlib inline

In [ ]:
import itertools
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# PHYS 395 - week 8

**Matt Wiens - #301294492**

This notebook will be organized similarly to the lab script, with major headings corresponding to the headings on the lab script.

*The TA's name (Ignacio) will be shortened to "IC" whenever used.*

## Setup 

In [ ]:
# Set default plot size
plt.rcParams["figure.figsize"] = (12, 9)

In [ ]:
%%javascript
IPython.OutputArea.auto_scroll_threshold = 9999

# Simulated annealing

We will start by working through some problems involving simulated annealing.

## Finding the global minimum of 1D energy function

Here we want to use annealing to find the global minimum of

\begin{equation}
    E(x) = x^2 - 2 \cos \left(2 \pi x \right)
    .
\end{equation}

First, let's plot this function.

In [ ]:
# First define the function
energy_fn = lambda x: x ** 2 - 2 * np.cos(2 * np.pi * x)

In [ ]:
# Now plot
_, ax = plt.subplots()

xs = np.linspace(-5, 5, 500)
Es = energy_fn(xs)

ax.plot(xs, Es)

ax.set_xlabel("$x$")
ax.set_ylabel("$E$");

Now we'll use simulated annealing to find the global minimum. First we'll define the parameters for our method.

In [ ]:
# Initial and final temperatures
T_init = 1.0
T_end = 0.01

# Decay constant
tau = 100

# Initial x position -- this could be anything really
x_init = 3

To find the number of iterations required, note that we can invert the formula

\begin{equation}
    T = T_0 \exp \left(- \frac{t}{\tau} \right)
\end{equation}

to get the time parameter $t$ as a function of $T$; i.e.,

\begin{equation}
    t = \tau \log \left( \frac{T_0}{T} \right)
    .
\end{equation}

Then, we can find the number of iterations required by plugging in the final temperature.

In [ ]:
# Calculate how many iterations given initial and final
# temperatures
num_iterations = int(np.ceil(tau * np.log(T_init / T_end))) + 1

# Set up arrays to track data
energies = np.zeros(num_iterations)
positions = np.zeros(num_iterations)

energies[0] = energy_fn(x_init)
positions[0] = x_init

# Set up temperatures to iterate over
temps = T_init * np.exp(-np.arange(num_iterations) / tau)

# Now run the simulation
for i, T in enumerate(temps[1:], 1):
    # Generate a candidate position
    new_x = positions[i - 1] + np.random.normal()
    new_E = energy_fn(new_x)
    
    # See if move accepted
    dE = new_E - energies[i - 1]
    
    if np.exp(-dE / T) >= np.random.rand():
        # Move accepted
        positions[i] = new_x
        energies[i] = new_E
    else:
        positions[i] = positions[i - 1]
        energies[i] = energies[i - 1]

Now let's overlay the result of of simulation on the plot we generated above.

In [ ]:
# Now plot
_, ax = plt.subplots()

xs = np.linspace(-5, 5, 500)
Es = energy_fn(xs)

ax.plot(xs, Es)
ax.plot(positions[np.argmin(energies)], np.min(energies), ".", markersize=20)

ax.set_xlabel("$x$")
ax.set_ylabel("$E$")

ax.legend(["energy curve", "simulation global minimum"]);

The values we've used here lead to a good result. I've tried reducing the decay constant $\tau$ and the result was close to the minimum but not quite there (less iterations). Same idea for raising the final temperature. Anyway, the current values lead to a good result, and at least on my machine the simulation is fast, so I'm happy with it.

## Finite cluster shapes

Now we're going to consider a finite set of interacting particles, and find the minimum energy states by considering the Lennard-Jones potential. Recall that the Lennard-Jones potential is given by

\begin{equation}
    V_{LJ}(r) = 4 \epsilon \left[ \frac{1}{r^{12}} - \frac{1}{r^6} \right]
    ,
\end{equation}

where the distance $r$ is expressed in terms of particle diameter, and so the total energy of the system will be given by

\begin{equation}
    E = \sum_{i = 1}^N \sum_{j = i + 1}^N V_{LJ}(r_{ij})
    .
\end{equation}

To make this problem more tractable, we'll restrict the $N$ particles to a 2D box of size $L$ x $L$ and impose periodic boundary conditions. Let's set up parameters.

In [ ]:
# Number of particles
N = 10

# Length of box (in units of particle diameter)
L = 10

# Initial and final temperature
T_init = 1.5
T_end = 0.001

# Decay constant
tau = 200

# Epsilon parameter
eps = 1.5

# Standard deviation of x and y position change
sigma_r = 0.5

Now let's set up functions for calculating the total energy of the system, and changes in energy.

In [ ]:
def v_lj(r: float, eps: float = eps) -> float:
    return 4 * eps * (1 / r ** 12 - 1 / r ** 6)


def get_total_energy(positions: np.ndarray, N: int = N) -> float:
    total = 0

    for i, j in itertools.combinations(range(N), 2):
        r = np.linalg.norm(positions[i, :] - positions[j, :])
        total += v_lj(r)

    return total


def get_change_in_energy(
    old_positions: np.ndarray, new_position: np.ndarray, row_idx: int, N: int = N
) -> float:
    total = 0

    for i in range(N):
        if i == row_idx:
            continue

        r = np.linalg.norm(new_position - old_positions[i, :])
        total += v_lj(r)

    return total

Now we'll simulate this system.

In [ ]:
# First let's randomly pick positions for the N particles
init_positions = L * np.random.rand(N, 2)

# Calculate how many iterations given initial and final
# temperatures
num_iterations = int(np.ceil(tau * np.log(T_init / T_end))) + 1

# Set up temperatures to iterate over
temps = T_init * np.exp(-np.arange(num_iterations) / tau)

# Now set up arrays for our data
all_positions = np.zeros((num_iterations, N, 2))
energies = np.zeros(num_iterations)

all_positions[0, :, :] = init_positions
energies[0] = get_total_energy(init_positions)

# Now run the simulation
for t, T in enumerate(temps[1:], 1):
    current_positions = np.copy(all_positions[t - 1, :, :])
    current_energy = energies[t - 1]

    # Perform a sweep over all N particles
    for i in range(N):
        # Get candidate position for ith particle
        new_pos = current_positions[i, :] + np.random.normal(0, sigma_r, 2)
        new_pos = new_pos - L * np.floor(new_pos / L)
        
        # Decide whether to accept it
        dE = get_change_in_energy(current_positions, new_pos, i)

        if np.exp(-dE / T) >= np.random.rand():
            # Move accepted
            current_positions[i, :] = new_pos
            current_energy += dE
            
    # Capture sample
    all_positions[t, :, :] = current_positions
    energies[t] = current_energy

Now we'll plot the four lowest energy states from our sample.

In [ ]:
# Get the indices of the k lowest energy states
k = 4

indices = np.argpartition(energies, k)[:k]

In [ ]:
_, axes = plt.subplots(2, 2, sharex=True, sharey=True)

for idx, ax in enumerate(axes.flatten()):
    ax.plot(all_positions[indices[idx], :, 0], all_positions[indices[idx], :, 1], "*")

## Number partitioning

For this problem we have $N$ positive numbers that we want to divide as equally as possible into two groups. Let $s_i = 1$ if the $i$th number $n_i$ belongs to the first group; else $s_i = - 1$. We thus want to minimize

\begin{equation}
    H = \left( \sum_{i = 1}^N n_i s_i \right)^2
    .
\end{equation}

Note that we square the sum because the most optimal values of $H$ are those that are close to zero as possible, regardless of sign.

As we've done before, we'll first set up constants, and then run the simulation.

In [ ]:
# Determine which/how many numbers are in the problem
N = 15
n_min = 1
n_max = 100

# Initial and final temperature
T_init = 1.5
T_end = 0.001

# Decay constant
tau = 200

# The energy function to minimize
energy_fn = lambda ns, ss: (ns @ ss) ** 2

In [ ]:
# Randomly pick the n-values and the initial s-values
n_vals = np.random.randint(n_min, n_max + 1, N)
init_s_vals = np.random.choice([-1, 1], N)

# Calculate how many iterations given initial and final
# temperatures
num_iterations = int(np.ceil(tau * np.log(T_init / T_end))) + 1

# Set up temperatures to iterate over
temps = T_init * np.exp(-np.arange(num_iterations) / tau)

# Now set up arrays for our data
s_vals = np.zeros((num_iterations, N))
energies = np.zeros(num_iterations)

s_vals[0, :] = init_s_vals
energies[0] = energy_fn(n_vals, init_s_vals)

# Now run the simulation
for t, T in enumerate(temps[1:], 1):
    current_s_vals = np.copy(s_vals[t - 1, :])
    current_energy = energies[t - 1]

    # Perform a sweep over all N particles
    for _ in range(N):
        # Randomly select one of the "spins" to flip
        spin_idx = np.random.randint(0, N)

        proposed_s_vals = np.copy(current_s_vals)
        proposed_s_vals[spin_idx] = -proposed_s_vals[spin_idx]

        # Decide whether to accept it
        dE = energy_fn(n_vals, proposed_s_vals) - current_energy

        if np.exp(-dE / T) >= np.random.rand():
            # Move accepted
            current_s_vals = proposed_s_vals
            current_energy += dE

    # Capture sample
    s_vals[t, :] = current_s_vals
    energies[t] = current_energy

In [ ]:
print("n-vals: %s" % n_vals)
print("Minimum H: %s" % np.min(energies))

Whether or not we can get $H = 0$ is problem dependent (that is, it depends on the values of $n_i$), but all runs I've done for this problem converge to a single value (and quite quickly).

For example, if we have $N = 16$ and

\begin{equation}
    n_i = i
    ,
\end{equation}

then for this configuration we can reach $H = 0$. Additionally there are many ways to reach this optimal value, many of which are reached in the Metropolis simulation.

We demonstrate this below.

In [ ]:
# Determine which/how many numbers are in the problem
N = 16

# Initial and final temperature
T_init = 1.5
T_end = 0.001

# Decay constant
tau = 200

# The energy function to minimize
energy_fn = lambda ns, ss: (ns @ ss) ** 2

In [ ]:
# Set up the n-values
n_vals = np.zeros(N) + 1

# Randomly pick the initial s-values
init_s_vals = np.random.choice([-1, 1], N)

# Calculate how many iterations given initial and final
# temperatures
num_iterations = int(np.ceil(tau * np.log(T_init / T_end))) + 1

# Set up temperatures to iterate over
temps = T_init * np.exp(-np.arange(num_iterations) / tau)

# Now set up arrays for our data
s_vals = np.zeros((num_iterations, N))
energies = np.zeros(num_iterations)

s_vals[0, :] = init_s_vals
energies[0] = energy_fn(n_vals, init_s_vals)

# Now run the simulation
for t, T in enumerate(temps[1:], 1):
    current_s_vals = np.copy(s_vals[t - 1, :])
    current_energy = energies[t - 1]

    # Perform a sweep over all N particles
    for _ in range(N):
        # Randomly select one of the "spins" to flip
        spin_idx = np.random.randint(0, N)

        proposed_s_vals = np.copy(current_s_vals)
        proposed_s_vals[spin_idx] = -proposed_s_vals[spin_idx]

        # Decide whether to accept it
        dE = energy_fn(n_vals, proposed_s_vals) - current_energy

        if np.exp(-dE / T) >= np.random.rand():
            # Move accepted
            current_s_vals = proposed_s_vals
            current_energy += dE

    # Capture sample
    s_vals[t, :] = current_s_vals
    energies[t] = current_energy

In [ ]:
# Find number of unique zero-energy configurations
num_uniq = np.unique(s_vals[np.where(energies == 0), :], axis=1).shape[1]

print("Num unique zero-energy configurations found: %s" % num_uniq)

We can also write the energy function $H$ in another form:

\begin{align}
    H
        &= \left( \sum_{i = 1}^N n_i s_i \right)^2 \\
        &= \left( \sum_{i = 1}^N n_i s_i \right) \left( \sum_{i = 1}^N n_i s_i \right) \\
        &= \sum_{i = 1}^N \sum_{j = 1}^N n_i n_j s_i s_j \\
        &= \sum_{i = 1}^N \sum_{j = 1}^N J_{i, j} s_i s_j
        ,
\end{align}

where in the last step we set $J_{i, j} = n_i, n_j$.

Lets run our above simulation with random $n_i$ values using the rewritten from of $H$.

In [ ]:
# Determine which/how many numbers are in the problem
N = 15
n_min = 1
n_max = 100

# Initial and final temperature
T_init = 1.5
T_end = 0.001

# Decay constant
tau = 200

# The energy function to minimize
def energy_fn(ns: np.ndarray, ss: np.ndarray) -> int:
    total = 0
    
    for i, j in itertools.product(range(N), range(N)):
        total += ns[i] * ns[j] * ss[i] * ss[j]
    
    return total

In [ ]:
# Randomly pick the n-values and the initial s-values
n_vals = np.random.randint(n_min, n_max + 1, N)
init_s_vals = np.random.choice([-1, 1], N)

# Calculate how many iterations given initial and final
# temperatures
num_iterations = int(np.ceil(tau * np.log(T_init / T_end))) + 1

# Set up temperatures to iterate over
temps = T_init * np.exp(-np.arange(num_iterations) / tau)

# Now set up arrays for our data
s_vals = np.zeros((num_iterations, N))
energies = np.zeros(num_iterations)

s_vals[0, :] = init_s_vals
energies[0] = energy_fn(n_vals, init_s_vals)

# Now run the simulation
for t, T in enumerate(temps[1:], 1):
    current_s_vals = np.copy(s_vals[t - 1, :])
    current_energy = energies[t - 1]

    # Perform a sweep over all N particles
    for _ in range(N):
        # Randomly select one of the "spins" to flip
        spin_idx = np.random.randint(0, N)

        proposed_s_vals = np.copy(current_s_vals)
        proposed_s_vals[spin_idx] = -proposed_s_vals[spin_idx]

        # Decide whether to accept it
        dE = energy_fn(n_vals, proposed_s_vals) - current_energy

        if np.exp(-dE / T) >= np.random.rand():
            # Move accepted
            current_s_vals = proposed_s_vals
            current_energy += dE

    # Capture sample
    s_vals[t, :] = current_s_vals
    energies[t] = current_energy

In [ ]:
print("n-vals: %s" % n_vals)
print("Minimum H: %s" % np.min(energies))

This takes a bit longer due to the code used to evaluate the energy function, but other than that the results are identical (as they should be) to the simulations with the previous energy function expression.

## Traveling salesman problem

Now we'll use annealing to solve the traveling saleman problem.

Let's load the data we were given.

In [ ]:
cities_pos_small = np.loadtxt("cities_small.txt", delimiter=",")
cities_pos_large = np.loadtxt("cities_large.txt", delimiter=",")

Now we'll run simulated annealing code for both of these datasets.

In [ ]:
cities_data = cities_pos_small

In [ ]:
# Number of cities
N = len(cities_data)

# Precompute the distances between cities
distances = np.zeros((N, N))

for i, j in itertools.combinations(range(N), 2):
    dist = np.linalg.norm(cities_data[i, :] - cities_data[j, :])

    distances[i, j] = dist
    distances[j, i] = dist

# Initial and final temperature
T_init = 1.5
T_end = 0.001

# Decay constant
tau = 200

# The energy function to minimize
def energy_fn(path: np.ndarray) -> float:
    total = 0

    for i in range(1, N):
        total += distances[path[i], path[i - 1]]

    total += distances[path[0], path[N - 1]]

    return total

In [ ]:
# Randomly pick initial path
init_path = np.concatenate([[0], np.random.permutation(range(1, N)), [0]])

# Calculate how many iterations given initial and final
# temperatures
num_iterations = int(np.ceil(tau * np.log(T_init / T_end))) + 1

# Set up temperatures to iterate over
temps = T_init * np.exp(-np.arange(num_iterations) / tau)

# Now set up arrays for our data
paths = np.zeros((num_iterations, N + 1), dtype=int)
energies = np.zeros(num_iterations)

paths[0, :] = init_path
energies[0] = energy_fn(init_path)

# Now run the simulation
for t, T in enumerate(temps[1:], 1):
    path = np.copy(paths[t - 1, :])

    # Randomly interchange one of the cities
    city_1 = np.random.randint(1, N)
    city_2 = np.random.randint(1, N)

    while city_2 == city_1:
        city_2 = np.random.randint(1, N)

    path[city_1] = city_2
    path[city_2] = city_1

    # Compute the energy
    energy = energy_fn(path)

    # Decide whether to accept this change
    dE = energy - energies[t - 1]

    if np.exp(-dE / T) >= np.random.rand():
        # Move accepted
        paths[t, :] = path
        energies[t] = energy
    else:
        paths[t, :] = paths[t - 1, :]
        energies[t] = energies[t - 1]

In [ ]:
# Let's print out all final paths that achieve
# the minimum energy (there likely is just one)
min_energy = np.min(energies)

best_paths = (np.unique(paths[np.where(energies == min_energy), :], axis=1))[0]

In [ ]:
print("All best paths: %s" % best_paths)

We see there's just one optimal path picked up by the simulation.

Now we'll try the same thing with the larger dataset.

In [ ]:
cities_data = cities_pos_large

In [ ]:
# Number of cities
N = len(cities_data)

# Precompute the distances between cities
distances = np.zeros((N, N))

for i, j in itertools.combinations(range(N), 2):
    dist = np.linalg.norm(cities_data[i, :] - cities_data[j, :])

    distances[i, j] = dist
    distances[j, i] = dist

# Initial and final temperature
T_init = 1.5
T_end = 0.001

# Decay constant
tau = 200

# The energy function to minimize
def energy_fn(path: np.ndarray) -> float:
    total = 0

    for i in range(1, N):
        total += distances[path[i], path[i - 1]]

    total += distances[path[0], path[N - 1]]

    return total

In [ ]:
# Randomly pick initial path
init_path = np.concatenate([[0], np.random.permutation(range(1, N)), [0]])

# Calculate how many iterations given initial and final
# temperatures
num_iterations = int(np.ceil(tau * np.log(T_init / T_end))) + 1

# Set up temperatures to iterate over
temps = T_init * np.exp(-np.arange(num_iterations) / tau)

# Now set up arrays for our data
paths = np.zeros((num_iterations, N + 1), dtype=int)
energies = np.zeros(num_iterations)

paths[0, :] = init_path
energies[0] = energy_fn(init_path)

# Now run the simulation
for t, T in enumerate(temps[1:], 1):
    path = np.copy(paths[t - 1, :])

    # Randomly interchange one of the cities
    city_1 = np.random.randint(1, N)
    city_2 = np.random.randint(1, N)

    while city_2 == city_1:
        city_2 = np.random.randint(1, N)

    path[city_1] = city_2
    path[city_2] = city_1

    # Compute the energy
    energy = energy_fn(path)

    # Decide whether to accept this change
    dE = energy - energies[t - 1]

    if np.exp(-dE / T) >= np.random.rand():
        # Move accepted
        paths[t, :] = path
        energies[t] = energy
    else:
        paths[t, :] = paths[t - 1, :]
        energies[t] = energies[t - 1]

In [ ]:
# Let's print out all final paths that achieve
# the minimum energy (there likely is just one)
min_energy = np.min(energies)

best_paths = (np.unique(paths[np.where(energies == min_energy), :], axis=1))[0]

In [ ]:
print("All best paths: %s" % best_paths)

Again, the simulation picked up exactly one optimal path.